# Install KB-LAB client

```bash
pipenv install kblab-client
```

In [24]:
import os
from kblab import Archive
from json import load


a = Archive('https://betalab.kb.se/', auth=(os.environ['KBLAB_USER'], os.environ['KBLAB_PASSWD']))

query = { "tags": "issue", "meta.created": "1927" }
query = { "tags": "protokoll" }

a.search(query)


Result(start='0', n='2960', m='2960', keys=<generator object HttpArchive._search_iter at 0x7fe87490df50>, hits=<generator object package_generator at 0x7fe8747da2d0>)

In [29]:
result = a.search({ "tags": "protokoll" }, max=2)

for package_id in result:

    package = a.get(package_id)

    print(dir(package))
    print(package.list())
    
    for file in package:
        content = package.get_raw(file).read()
        
        print(dir(content))
        print(dir(file))

        break
    break



['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_desc', '_get_iter', '_mode', '_reload', 'auth', 'base', 'close', 'description', 'get_iter', 'get_location', 'get_raw', 'label', 'list', 'read', 'server_base', 'status', 'url']
['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__sub

In [31]:
print(dir(content))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'center', 'count', 'decode', 'endswith', 'expandtabs', 'find', 'fromhex', 'hex', 'index', 'isalnum', 'isalpha', 'isascii', 'isdigit', 'islower', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


In [22]:
print(package.url)

# ['__class__',
#  'auth',
#  'base',
#  'close',
#  'description',
#  'get_iter',
#  'get_location',
#  'get_raw',
#  'label',
#  'list',
#  'read',
#  'server_base',
#  'status',
#  'url']


https://betalab.kb.se/prop-198990--99/


In [ ]:
#dir(result)
#[x for x in result]

#import json
#json.dumps(query)

#from urllib.parse import urlencode
#urlencode(query)

In [ ]:

for package_id in a.search({ 'label': 'AFTONBLADET' }, max=2):
    print(package_id)
    p = a.get(package_id)

    #print(p.label)



In [3]:
import os
import zipfile
import fnmatch
import re

from kblab import Archive
#from json import load

# from utility import alto2text
import xmltodict

def kblab_connect():

    kb_archive = Archive('https://betalab.kb.se/', auth=(os.environ['KBLAB_USER'], os.environ['KBLAB_PASSWD']))

    return kb_archive

def kblab_download_package(kb_archive, package_id, target_archive, excludes=None):

    excludes = excludes or []

    package = kb_archive.get(package_id)

    if 'content.json' not in package:
        print("warning: {} has no content".format(package_id))
        return

    text_items = []
    text_item_id = -1
    for filename in sorted(package.list()):

        if any(fnmatch.fnmatch(filename, pattern) for pattern in excludes):
            # if os.path.splitext(filename)[1] in excludes:
            print("skipping: {}".format(filename))
            continue

        content = package.get_raw(filename).read()

        target_archive.writestr(os.path.join(package_id, filename), content, zipfile.ZIP_DEFLATED)

        print("stored: {}".format(filename))

        m = re.match(r'prot_(\d{4})__(\d+)\-(\d+)\.xml', filename)
        if m is not None:

            year, id, page = m.groups()

            assert package_id == "prot-{}--{}".format(year, id)

            if (text_item_id + 1) != int(page):
                print("warning: page(s) with no XML found {} expected {}".format(int(page), previous_page + 1))

            page_content = xmltodict.parse(content)
            assert isinstance(page_content, dict) and 'alto' in page_content
            package_documents.append(page_content)

    #if len(pages) > 0:
    #    target_archive.writestr(os.path.join(package_id, "{}.txt".format(package_id)), '\n'.join(pages), zipfile.ZIP_DEFLATED)

    return package_documents

kb_archive = kblab_connect()
target_folder = "/home/roger/tmp"
target_filename = os.path.join(target_folder, "riksdagens_protokoll.zip")
query = { "tags": "protokoll" }
max_count = 2

with zipfile.ZipFile(target_filename, "w") as target_archive:

    for package_id in kb_archive.search(query, max=max_count):

        package = kb_archive.get(package_id)

        package_documents = kblab_download_package(kb_archive, package_id, target_archive, ["*.jpg", "*.jb2e"])



stored: content.json
stored: coverage.json
stored: coverage.txt
stored: meta.json
skipping: prot_1972__145-000.jpg
stored: prot_1972__145-000.xml
skipping: prot_1972__145-001.jpg
stored: prot_1972__145-001.xml
skipping: prot_1972__145-002.jpg
stored: prot_1972__145-002.xml
skipping: prot_1972__145-003.jpg
stored: prot_1972__145-003.xml
skipping: prot_1972__145-004.jb2e
skipping: prot_1972__145-005.jpg
stored: prot_1972__145-005.xml
skipping: prot_1972__145-006.jb2e
skipping: prot_1972__145-007.jpg
stored: prot_1972__145-007.xml
skipping: prot_1972__145-008.jb2e
skipping: prot_1972__145-009.jpg
stored: prot_1972__145-009.xml
skipping: prot_1972__145-010.jb2e
skipping: prot_1972__145-011.jpg
stored: prot_1972__145-011.xml
skipping: prot_1972__145-012.jpg
stored: prot_1972__145-012.xml
skipping: prot_1972__145-013.jpg
stored: prot_1972__145-013.xml
skipping: prot_1972__145-014.jpg
stored: prot_1972__145-014.xml
skipping: prot_1972__145-015.jpg
stored: prot_1972__145-015.xml
skipping: prot

In [8]:
document = pages[0]
layout = page['alto']['Layout']
for page in layout:
    pass

In [12]:
dir(package)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_desc',
 '_get_iter',
 '_mode',
 '_reload',
 'auth',
 'base',
 'close',
 'description',
 'get_iter',
 'get_location',
 'get_raw',
 'label',
 'list',
 'read',
 'server_base',
 'status',
 'url']

In [14]:
dir(package.description)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']